In [1]:
#Loading in appropriate packages
import numpy as np, matplotlib.pyplot as plt
import pandas as pd
import requests
import time
import json
from bs4 import BeautifulSoup
import sys
from scipy import stats
from sklearn.cross_validation import *

pd.set_option('display.width', 500)
pd.set_option('display.max_columns', 100)
pd.set_option('display.notebook_repr_html', True)
import seaborn as sns
sns.set_style("whitegrid")
sns.set_context("poster")

%matplotlib inline

In [2]:
#Loading in Relevant Dataframes
master = pd.read_csv('data/lahman/Master.csv')
pitching = pd.read_csv('data/lahman/Pitching.csv')
smalldf = pd.read_csv('data/small.csv')

For details on the Lahman file columns, see "2.3: Pitching Table": http://seanlahman.com/files/database/readme2014.txt

pitcher_sums is just the "sums", so be sure to recalculate things like ERA and BAOpp that don't add linearly

In [3]:
#Ben's intro code
retro_to_lah = dict(zip(master['retroID'], master['playerID']))
#retro_to_hand = dict(zip(master['retroID'], master['throws']))
#pitcherIDs = smalldf['pID'].unique()#RetroIDs
#L_pitcherIDs = [retro_to_lah[name] for name in pitcherIDs] #LahmanIDs
#grouped =  pitching.groupby('playerID').sum().reset_index()
#mask = np.in1d(grouped.playerID, L_pitcherIDs)
#pitcher_sums = grouped[mask]

In [4]:
smalldf.head()

,bID,pID,AB,H,PA,RL,SAC,SO,TB,W,matchID,AVG,OBP,ov_AB,ov_H,ov_PA,ov_SAC,ov_SO,ov_TB,ov_W,ov_AVG,ov_OBP,ov_FACED,ov_SO_PCT,ov_W_PCT,ov_H_PCT,ovp_AB,ovp_H,ovp_PA,ovp_SAC,ovp_SO,ovp_TB,ovp_W,ovp_AVG,ovp_OBP,ovp_FACED,ovp_SO_PCT,ovp_W_PCT,ovp_H_PCT
0,schic002,benea001,9,0,9,R,1,4,0,0,schic002_benea001,0.000000,0.000000,146,22,150,14,57,30,4,0.150685,0.173333,14,0.38,0.026667,0.146667,3046,827,3425,46,609,1713,379,0.271504,0.352117,136,0.177810,0.110657,0.241460
1,schic002,browk001,11,1,11,R,0,6,1,0,schic002_browk001,0.090909,0.090909,146,22,150,14,57,30,4,0.150685,0.173333,14,0.38,0.026667,0.146667,4582,1159,4989,47,892,1974,407,0.252946,0.313891,222,0.178793,0.081579,0.232311
2,schic002,fassj001,11,2,11,L,1,5,4,0,schic002_fassj001,0.181818,0.181818,146,22,150,14,57,30,4,0.150685,0.173333,14,0.38,0.026667,0.146667,3006,847,3357,40,582,1587,351,0.281770,0.356866,185,0.173369,0.104558,0.252309
3,schic002,glavt001,14,3,14,L,1,3,5,0,schic002_glavt001,0.214286,0.214286,146,22,150,14,57,30,4,0.150685,0.173333,14,0.38,0.026667,0.146667,7561,2115,8413,102,985,3852,852,0.279725,0.352668,280,0.117081,0.101272,0.251397
4,schic002,hampm001,16,2,16,L,1,6,2,0,schic002_hampm001,0.125000,0.125000,146,22,150,14,57,30,4,0.150685,0.173333,14,0.38,0.026667,0.146667,4782,1361,5339,53,666,2355,557,0.284609,0.359243,232,0.124742,0.104327,0.254917


In [5]:
pitching.head()

,playerID,yearID,stint,teamID,lgID,W,L,G,GS,CG,SHO,SV,IPouts,H,ER,HR,BB,SO,BAOpp,ERA,IBB,WP,HBP,BK,BFP,GF,R,SH,SF,GIDP
0,bechtge01,1871,1,PH1,NaN,1,2,3,3,2,0,0,78,43,23,0,11,1,NaN,7.96,NaN,NaN,NaN,0,NaN,NaN,42,NaN,NaN,NaN
1,brainas01,1871,1,WS3,NaN,12,15,30,30,30,0,0,792,361,132,4,37,13,NaN,4.50,NaN,NaN,NaN,0,NaN,NaN,292,NaN,NaN,NaN
2,fergubo01,1871,1,NY2,NaN,0,0,1,0,0,0,0,3,8,3,0,0,0,NaN,27.00,NaN,NaN,NaN,0,NaN,NaN,9,NaN,NaN,NaN
3,fishech01,1871,1,RC1,NaN,4,16,24,24,22,1,0,639,295,103,3,31,15,NaN,4.35,NaN,NaN,NaN,0,NaN,NaN,257,NaN,NaN,NaN
4,fleetfr01,1871,1,NY2,NaN,0,1,1,1,1,0,0,27,20,10,0,3,0,NaN,10.00,NaN,NaN,NaN,0,NaN,NaN,21,NaN,NaN,NaN


#Pitchers Similarity Formula

Start with a thousand and then subtract the following:

One point for each difference of 1 win.

One point for each difference of 2 losses.

One point for each difference of .002 in winning percentage (max 100 points).

One point for each difference of .02 in ERA (max 100 points).

One point for each difference of 10 games pitched.

One point for each difference of 20 starts.

One point for each difference of 20 complete games.

One point for each difference of 50 innings pitched.

One point for each difference of 50 hits allowed.

One point for each difference of 30 strikeouts.

One point for each difference of 10 walks.

One point for each difference of 5 shutouts.

One point for each difference of 3 saves.

If they throw with a different hand and are starters subtract 10, relievers 25. For relievers you halve the winning percentage penalty. For all pitchers, the winning percentage penalty can be no larger than 1.5 times the wins and losses penalty. Relievers are defined as more relief appearances than starts and less than 4.00 innings per appearance.

We plugged all this in to create the lists you see on the player pages. Note that a player must have 100 innings pitched or 500 at bats before being considered, and to be truly accurate you need to look at whole careers, but it is fun to speculate all the same.

In [6]:
##input is the pitcherIDs
def pitcher_sim_alt(p1name, p2name, pitching, same_hand=True):
    listed= [p1name,p2name]
    two_pitcherIDs= [retro_to_lah[name] for name in listed] #getting the lahman ids for the 2 pitcher ids to use in Pitching Dataframe

    grouped2 =  pitching.groupby('playerID').sum().reset_index() #getting the summed data for each unique playerID
    mask2 = np.in1d(grouped2.playerID, two_pitcherIDs) #creating a mask for the 2 pitcher IDs needed
    pitchers=grouped2[mask2].reset_index() #resetting the index to 0,1 to make concatenating dataframes easier
    pitchers.ERA = 9.*(pitchers.ER)/(pitchers.IPouts / 3.) #generating ERA 
    pitchers.BAOpp = (pitchers.H + pitchers.BB + pitchers.IBB + pitchers.HBP) / pitchers.BFP #batting average against
    pitchers['WiP'] = pitchers.W/pitchers.G #win percentage
    pitchers['IP'] = pitchers.IPouts/3. #innings pitched
    
    #determining whether a pitcher is a relief pitcher based on whether more games started than not 
    #and less than 4 innings pitched per game
    pitchers['Games']=pitchers['GS']-pitchers['G']/2. #if positive, more games started than games in relief
    pitchers['inningsper']=pitchers['IP']/pitchers['G'] #innings per game 
    relief=[]
    for i in pitchers.iterrows():
        if i[1]['Games']<0. and i[1]['inningsper']<4.:
            relief.append(1.)
        else:
            relief.append(0.)
    dfrel=pd.DataFrame({'REL':relief})
    
    #adding handedness and whether a pitcher is a relief pitcher to the dataframe 
    final=pitchers.join([dfhands,dfrel], how='outer')
    
    #getting the 2 rows from the final dataframe to perform calculations 
    p1=final[final.playerID==two_pitcherIDs[0]]
    p2=final[final.playerID==two_pitcherIDs[1]]
    
    
    #Wins and Losses Penalties
    win_diff=np.abs(int(p1['W'])-int(p2['W'])) #1 pt for each win
    loss_diff=np.abs(int(p1['L'])-int(p2['L']))/2. #1 pt each 2 losses
    
    #Winning Percentage Penalty
    if int(p1['REL'])==1:     #winning percentage is halved for relief pitchers
        p1['WiP']=p1['WiP']/2.
    if int(p2['REL'])==1:
        p2['WiP']=p2['WiP']/2.
    wip_diff=np.abs(float(p1['WiP'])-float(p2['WiP']))/.002 #1 pt diff of winning percentage of 0.002 
    if wip_diff > 1.5*(win_diff+loss_diff): #winning percentage cannot be more than 1.5 times the penalties for win/loss differences 
        wip_diff= 1.5*(win_diff+loss_diff)
    if wip_diff >100: #penalty for winning percentage has a max= 100
        wip_diff=100   
    era_diff=np.abs(float(p1['ERA'])-float(p2['ERA']))/0.02 #1 pt diff of ERA of 0.02
    if era_diff >100:
        era_diff=100 #max =100
    
    #Handedness Penalties conditional on both pitchers being relief pitchers or not 
    if list(p1['RL'])[0]!=list(p2['RL'])[0] and int(p1['REL'])==0 and int(p2['REL'])==0: #handedness is different, relief pitchers
        hand_diff=10. 
    elif list(p1['RL'])[0]!=list(p2['RL'])[0] and int(p1['REL'])==1 and int(p2['REL'])==1.: #handedness is different, starters
        hand_diff=25.  
    else:
        hand_diff=0.
    
    #Additional Penalties
    gp_diff=np.abs(int(p1['G'])-int(p2['G']))/10. #1 pt for diff of 10 games played
    st_diff=np.abs(int(p1['GS'])-int(p2['GS']))/20. #1 pt for diff of 20 games started
    cg_diff=np.abs(int(p1['CG'])-int(p2['CG']))/20. #1 pt for diff of 20 games completed
    ip_diff=np.abs(float(p1['IP'])-float(p2['IP']))/50. #1 pt for diff of 50 innings pitched
    h_diff=np.abs(int(p1['H'])-int(p2['H']))/50. #1 pt for diff of 50 hits allowed
    sp_diff=np.abs(int(p1['SO'])-int(p2['SO']))/30. #1 pt for diff of 30 strikeouts
    bb_diff=np.abs(int(p1['BB'])-int(p2['BB']))/50. #1 pt for diff of 10 walks
    sho_diff=np.abs(int(p1['SHO'])-int(p2['SHO']))/5. #1 pt for diff of 5 shutouts
    sv_diff=np.abs(int(p1['SV'])-int(p2['SV']))/3. #1 pt for diff of 3 saves
    

    score=1000 #generating starting score and subtracting penalties
    final_score=score-(win_diff+loss_diff+wip_diff+era_diff+gp_diff+st_diff+cg_diff+
                       ip_diff+h_diff+sp_diff+bb_diff+sho_diff+sv_diff+hand_diff)
    #print final_score

    return final_score

In [7]:
##input is the smalldf rows corresponding to the 2 pitcher ids
def pitcher_sim(p1_matchups, p2_matchups, n_common=0):
    p1name=list(p1_matchups['pID'][:1])[0]  #getting the IDs for both pitchers
    p2name=list(p2_matchups['pID'][:1])[0]
    dfhands=pd.DataFrame({'RL':[list(p1_matchups['RL'][:1])[0],list(p2_matchups['RL'][:1])[0]]}) #retrieving handedness from  smalldf
    listed= [p1name,p2name]
    two_pitcherIDs= [retro_to_lah[name] for name in listed] #getting the lahman ids for the 2 pitcher ids to use in Pitching Dataframe

    grouped2 =  pitching.groupby('playerID').sum().reset_index() #getting the summed data for each unique playerID
    mask2 = np.in1d(grouped2.playerID, two_pitcherIDs) #creating a mask for the 2 pitcher IDs needed
    pitchers=grouped2[mask2].reset_index() #resetting the index to 0,1 to make concatenating dataframes easier
    pitchers.ERA = 9.*(pitchers.ER)/(pitchers.IPouts / 3.) #generating ERA 
    pitchers.BAOpp = (pitchers.H + pitchers.BB + pitchers.IBB + pitchers.HBP) / pitchers.BFP #batting average against
    pitchers['WiP'] = pitchers.W/pitchers.G #win percentage
    pitchers['IP'] = pitchers.IPouts/3. #innings pitched
    
    #determining whether a pitcher is a relief pitcher based on whether more games started than not 
    #and less than 4 innings pitched per game
    pitchers['Games']=pitchers['GS']-pitchers['G']/2. #if positive, more games started than games in relief
    pitchers['inningsper']=pitchers['IP']/pitchers['G'] #innings per game 
    relief=[]
    for i in pitchers.iterrows():
        if i[1]['Games']<0. and i[1]['inningsper']<4.:
            relief.append(1.)
        else:
            relief.append(0.)
    dfrel=pd.DataFrame({'REL':relief})
    
    #adding handedness and whether a pitcher is a relief pitcher to the dataframe 
    final=pitchers.join([dfhands,dfrel], how='outer')
    
    #getting the 2 rows from the final dataframe to perform calculations 
    p1=final[final.playerID==two_pitcherIDs[0]]
    p2=final[final.playerID==two_pitcherIDs[1]]
    
    
    #Wins and Losses Penalties
    win_diff=np.abs(int(p1['W'])-int(p2['W'])) #1 pt for each win
    loss_diff=np.abs(int(p1['L'])-int(p2['L']))/2. #1 pt each 2 losses
    
    #Winning Percentage Penalty
    if int(p1['REL'])==1:     #winning percentage is halved for relief pitchers
        p1['WiP']=p1['WiP']/2.
    if int(p2['REL'])==1:
        p2['WiP']=p2['WiP']/2.
    wip_diff=np.abs(float(p1['WiP'])-float(p2['WiP']))/.002 #1 pt diff of winning percentage of 0.002 
    if wip_diff > 1.5*(win_diff+loss_diff): #winning percentage cannot be more than 1.5 times the penalties for win/loss differences 
        wip_diff= 1.5*(win_diff+loss_diff)
    if wip_diff >100: #penalty for winning percentage has a max= 100
        wip_diff=100   
    era_diff=np.abs(float(p1['ERA'])-float(p2['ERA']))/0.02 #1 pt diff of ERA of 0.02
    if era_diff >100:
        era_diff=100 #max =100
    
    #Handedness Penalties conditional on both pitchers being relief pitchers or not 
    if list(p1['RL'])[0]!=list(p2['RL'])[0] and int(p1['REL'])==0 and int(p2['REL'])==0: #handedness is different, relief pitchers
        hand_diff=10. 
    elif list(p1['RL'])[0]!=list(p2['RL'])[0] and int(p1['REL'])==1 and int(p2['REL'])==1.: #handedness is different, starters
        hand_diff=25.  
    else:
        hand_diff=0.
    
    #Additional Penalties
    gp_diff=np.abs(int(p1['G'])-int(p2['G']))/10. #1 pt for diff of 10 games played
    st_diff=np.abs(int(p1['GS'])-int(p2['GS']))/20. #1 pt for diff of 20 games started
    cg_diff=np.abs(int(p1['CG'])-int(p2['CG']))/20. #1 pt for diff of 20 games completed
    ip_diff=np.abs(float(p1['IP'])-float(p2['IP']))/50. #1 pt for diff of 50 innings pitched
    h_diff=np.abs(int(p1['H'])-int(p2['H']))/50. #1 pt for diff of 50 hits allowed
    sp_diff=np.abs(int(p1['SO'])-int(p2['SO']))/30. #1 pt for diff of 30 strikeouts
    bb_diff=np.abs(int(p1['BB'])-int(p2['BB']))/50. #1 pt for diff of 10 walks
    sho_diff=np.abs(int(p1['SHO'])-int(p2['SHO']))/5. #1 pt for diff of 5 shutouts
    sv_diff=np.abs(int(p1['SV'])-int(p2['SV']))/3. #1 pt for diff of 3 saves
    

    score=1000 #generating starting score and subtracting penalties
    final_score=score-(win_diff+loss_diff+wip_diff+era_diff+gp_diff+st_diff+cg_diff+
                       ip_diff+h_diff+sp_diff+bb_diff+sho_diff+sv_diff+hand_diff)
    #print final_score

    return final_score

In [11]:
a, b = np.random.choice(smalldf.pID.unique(), size=2)

In [14]:
%%time
for i in np.arange(1000):
    p1, p2 = np.random.choice(smalldf.pID.unique(), size=2)
    pitcher_sim(smalldf[smalldf.pID==p1],smalldf[smalldf.pID==p2])

CPU times: user 1min 32s, sys: 5.03 s, total: 1min 37s
Wall time: 1min 38s


/Users/bcook/anaconda/lib/python2.7/site-packages/IPython/kernel/__main__.py:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/bcook/anaconda/lib/python2.7/site-packages/IPython/kernel/__main__.py:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


#Using Mapreduce to see if this works

In [62]:
def get_restaurant_reviews(pID, df, set_of_batters):
    """
    given a pitcher id and a set of batters, return the sub-dataframe of their
    averages.
    """
    mask = (df.bID.isin(set_of_batters)) & (df.pID==pID)
    avgs = df[mask]
    avgs = avgs[avgs.bID.duplicated()==False]
    return avgs

In [63]:
print smalldf.shape,
smalldf[smalldf.ov_FACED > 100].shape

(109688, 39)

(88234, 39)

In [64]:
trainlist=[]
testlist=[]
validatelist=[]
take=21 #21 matchups between validation and test set
for k, v in smalldf.groupby('bID'):
    if len(v) > 100: #batter has faced at least 150 pitchers
        train_rows, test_valid_rows = train_test_split(v, test_size=take)
        trainlist.append(train_rows)
        valid_rows, test_rows = train_test_split(test_valid_rows, test_size=0.4)
        validatelist.append(valid_rows) 
        testlist.append(test_rows) 
    else:
        trainlist.append(v)
traindf=pd.concat(trainlist)
validatedf=pd.concat(validatelist)
testdf=pd.concat(testlist)
print traindf.shape, validatedf.shape, testdf.shape

(101036, 39) (4944, 39) (3708, 39)


In [65]:
#Make sure each pitcher ID was encountered in training set
maskval= np.in1d(validatedf.pID, traindf.pID) 
masktest = np.in1d(testdf.pID, traindf.pID)
print np.sum(~maskval), np.sum(~masktest)

0 0


In [66]:
def recompute_frame(ldf):
    """
    takes a dataframe ldf, makes a copy of it, and returns the copy
    with all conglomerations recomputed
    this is used when a frame is subsetted.
    """
    ldfb=ldf.groupby('bID')
    ldfp=ldf.groupby('pID')
    nldf=ldf.copy()
    
    #Conglomerate pitcher stats
    nldf.set_index(['pID'], inplace=True)
    for col in ['AB', 'PA', 'H', 'TB', 'SAC', 'SO', 'W']:
        nldf['ovp_'+col] = ldfp[col].sum()
    nldf['ovp_AVG'] = nldf['ovp_H']/nldf['ovp_AB']
    nldf['ovp_FACED']= ldfp.AB.count()
    nldf['ovp_OBP'] = (nldf['ovp_H'] + nldf['ovp_W'])/nldf['ovp_PA']
    for col in ['SO', 'W', 'H']:
        nldf['ovp_' + col + '_PCT'] = nldf['ovp_' + col] / nldf['ovp_PA']
    nldf.reset_index(inplace=True)
    
    #Conglomerate batter stats
    nldf.set_index(['bID'], inplace=True)
    for col in ['AB', 'PA', 'H', 'TB', 'SAC', 'SO', 'W']:
        nldf['ov_'+col] = ldfb[col].sum()
    nldf['ov_AVG'] = nldf['ov_H']/nldf['ov_AB']
    nldf['ov_FACED']= ldfb.AB.count()
    nldf['ov_OBP'] = (nldf['ov_H'] + nldf['ov_W'])/nldf['ov_PA']
    for col in ['SO', 'W', 'H']:
        nldf['ov_' + col + '_PCT'] = nldf['ov_' + col] / nldf['ov_PA']
    nldf.reset_index(inplace=True)
    return nldf

In [67]:
traindf=recompute_frame(traindf)
validatedf=recompute_frame(validatedf)
testdf=recompute_frame(testdf)
validatedf=validatedf[['bID', 'pID','AVG']]
testdf=testdf[['bID', 'pID', 'AVG']]
traindf.head()

,bID,pID,AB,H,PA,RL,SAC,SO,TB,W,matchID,AVG,OBP,ov_AB,ov_H,ov_PA,ov_SAC,ov_SO,ov_TB,ov_W,ov_AVG,ov_OBP,ov_FACED,ov_SO_PCT,ov_W_PCT,ov_H_PCT,ovp_AB,ovp_H,ovp_PA,ovp_SAC,ovp_SO,ovp_TB,ovp_W,ovp_AVG,ovp_OBP,ovp_FACED,ovp_SO_PCT,ovp_W_PCT,ovp_H_PCT
0,aberb001,bellr003,9,1,10,R,0,0,1,1,aberb001_bellr003,0.111111,0.200000,359,78,385,4,41,120,26,0.21727,0.27013,30,0.106494,0.067532,0.202597,1340,416,1510,20,200,957,170,0.310448,0.388079,118,0.132450,0.112583,0.275497
1,aberb001,buehm001,13,2,14,L,0,3,6,1,aberb001_buehm001,0.153846,0.214286,359,78,385,4,41,120,26,0.21727,0.27013,30,0.106494,0.067532,0.202597,8822,2439,9460,115,1280,4535,638,0.276468,0.325264,410,0.135307,0.067442,0.257822
2,aberb001,burkj001,8,2,9,R,0,1,2,1,aberb001_burkj001,0.250000,0.333333,359,78,385,4,41,120,26,0.21727,0.27013,30,0.106494,0.067532,0.202597,3906,1122,4261,43,663,2140,355,0.287250,0.346632,209,0.155597,0.083314,0.263318
3,aberb001,castf001,7,1,11,R,0,0,1,4,aberb001_castf001,0.142857,0.454545,359,78,385,4,41,120,26,0.21727,0.27013,30,0.106494,0.067532,0.202597,2157,608,2388,29,344,1206,231,0.281873,0.351340,144,0.144054,0.096734,0.254606
4,aberb001,clemr001,25,7,26,R,1,2,9,1,aberb001_clemr001,0.280000,0.307692,359,78,385,4,41,120,26,0.21727,0.27013,30,0.106494,0.067532,0.202597,6103,1493,6774,60,1477,2796,671,0.244634,0.319457,263,0.218040,0.099055,0.220402


In [68]:
def compute_supports(df, upids):
    ubids=df.bID.unique()
    pitch = df.groupby('pID').bID.unique()
    bdict={}
    for e,v in zip(pitch.index.values, pitch.values):
        bdict[e] = np.array([item in v for item in ubids])
    pitchers=upids
    supports=[]
    supports_matrix = [[[] for i in range(len(pitchers))] for j in range(len(pitchers))]
    for i,p1 in enumerate(pitchers):
        for j,p2 in enumerate(pitchers):
            if  i < j:
                common_batters = set(pitch[p1]).intersection(set(pitch[p2]))
                n = len(common_batters)
                supports.append(n)
                supports_matrix[i][j] = common_batters
                supports_matrix[j][i] = common_batters
    print "mean support",np.mean(supports), "median support", np.median(supports)
    return supports, bdict, supports_matrix

In [69]:
ubids=traindf.bID.unique()#unique-user-ids
upids=traindf.pID.unique()#unique-item-ids
ubidmap={v:k for k,v in enumerate(ubids)}#of length U
upidmap={v:k for k,v in enumerate(upids)}#of length M

In [70]:
#for each restaurant get the unique userids who rated it
# pitch = traindf.groupby('pID').bID.unique()
# restaurants=upids
# lres=len(restaurants)
# supports=[[[] for i in range(lres)] for i in range(lres)]
# supporthistlist=[]
# for i,rest1 in enumerate(restaurants):
#     for j,rest2 in enumerate(restaurants):
#         if  i <= j:#its symmetric
#             if rest1==rest2:
#                 common_reviewers=pitch[rest1]
#             else:
#                 common_reviewers = set(pitch[rest1]).intersection(set(pitch[rest2]))
#                 supporthistlist.append(len(common_reviewers))
#             supports[i][j]=common_reviewers
#             supports[j][i]=common_reviewers
# print "Mean and Median support is:",np.mean(supporthistlist), np.median(supporthistlist)

In [71]:
print "Number of Matchups",traindf.shape[0]
print "Number of Batters", traindf.bID.unique().shape[0]
print "Number of Pitchers", traindf.pID.unique().shape[0]

Number of Matchups 101036
Number of Batters 910
Number of Pitchers 1297


In [72]:
s,d, supports_matrix =compute_supports(traindf, upids)

mean support 11.593010223 median support 4.0


In [21]:
#import cPickle as pickle
# pickle.dump(supports_matrix,open('data/supports_matrix.p','wb'))

In [22]:
#supports_matrix = pickle.load(open('data/supports_matrix.p','rb'))

In [73]:
class Database:
    "A class representing a database of similarities and common supports"
    
    def __init__(self, rindexmap, supports):
        "the constructor, takes a map of restaurant id's to integers"
        database={}
        self.rindexmap=rindexmap
        self.supports=supports
        l_keys=len(self.rindexmap.keys())
        self.database_sim=np.zeros([l_keys,l_keys])
        self.database_sup=np.zeros([l_keys, l_keys], dtype=np.int)

    def set_supports(self, supports):
        self.supports=supports
        
    def get(self, b1, b2):
        "returns a tuple of similarity,common_support given two business ids"
        sim=self.database_sim[self.rindexmap[b1]][self.rindexmap[b2]]
        nsup=self.database_sup[self.rindexmap[b1]][self.rindexmap[b2]]
        return (sim, nsup)

In [74]:
# db=Database(upidmap, s)

In [75]:
def mapper1(row):
    return row[1], (row[2], row[12], row[34])

In [76]:
def combiner(items):
    indict={}
    for key, value in items:
        if not indict.has_key(key):
            indict[key]=[]
        indict[key].append(value)
    return indict.items()

In [77]:
def reducer1(the_input):
    bID, values = the_input
    avgs=[]
    for pID,AVG,ov_AVG in values:
        avgs.append((pID,(AVG, ov_AVG)))
    return bID, avgs

In [78]:
from itertools import combinations_with_replacement
def mapper2(list_input):
    nlist = []
    comb = list(combinations_with_replacement(list_input[1], 2))
    for item in comb:
        if item[0][0] > item[1][0]:
            biz_pair = item[1][0], item[0][0]
            star_pair = item[1][1], item[0][1]
        else:
            biz_pair = item[0][0], item[1][0]
            star_pair = item[0][1], item[1][1]
        tup = (biz_pair, star_pair)
        nlist.append(tup)
    return nlist

In [79]:
def combiner_list(itemslist):
    indict={}
    for items in itemslist:
        for key, value in items:
            if not indict.has_key(key):
                indict[key]=[]
            indict[key].append(value)
    return indict.items()

In [80]:
def reducer2(item_dict):
    p1_id = item_dict[0][0]
    p2_id = item_dict[0][1]
#     AVG_1 = [x[0][0] for x in item_dict[1]]
#     AVG_2 = [x[1][0] for x in item_dict[1]]
#     ov_AVG_1 = [x[0][1] for x in item_dict[1]]
#     ov_AVG_2 = [x[1][1] for x in item_dict[1]]
    n_common = len(item_dict[1])
    print('a')
    p1 = smalldf[smalldf.pID==p1_id]#{'ov_AVG': ov_AVG_1, 'AVG': AVG_1}
    p2 = smalldf[smalldf.pID==p1_id]#{'ov_AVG': ov_AVG_2, 'AVG': AVG_2}
    rho = pitcher_sim(p1, p2, n_common)
    return (p1_id, p2_id),(rho, n_common)

In [81]:
def map_reduce(tuples):
    mapped1=map(mapper1, tuples)
    combine1=combiner(mapped1)
    reduced1=reduce(lambda x,y: x + [reducer1(y)], combine1, [])
    mapped2=map(mapper2,reduced1)
    combine2=combiner_list(mapped2)
    output=reduce(lambda x,y: x + [reducer2(y)], combine2, [])
    return output

In [82]:
# %%time
# tuples=traindf.itertuples()
# sims2=map_reduce(tuples)

In [83]:
# import cPickle as pickle
# pickle.dump(sims2,open('sims2.p','wb'))
# #sims2 = pickle.load(open('sims2.p','rb'))

In [84]:
def populate_from_mr(db, df, mapredlist):
    for tpair,vpair in mapredlist:
        i1=db.rindexmap[tpair[0]]
        i2=db.rindexmap[tpair[1]]
        db.database_sim[i1][i2]=vpair[0]
        db.database_sup[i1][i2]=vpair[1]
        db.database_sim[i2][i1]=vpair[0]
        db.database_sup[i2][i1]=vpair[1]
 

In [85]:
# populate_from_mr(db, traindf, sims)

In [86]:
db2=Database(upidmap, supports_matrix)

In [87]:
def get_restaurant_reviews(pID, df, set_of_batters):
    """
    given a pitcher id and a set of batters, return the sub-dataframe of their
    averages.
    """
    mask = (df.bID.isin(set_of_batters)) & (df.pID==pID)
    avgs = df[mask]
    avgs = avgs[avgs.bID.duplicated()==False]
    return avgs

In [88]:
def calculate_similarity(db, df, p1, p2, similarity_func):
    # find common reviewers
    common_reviewers = db.supports[db.rindexmap[p1]][db.rindexmap[p2]]
    n_common=len(common_reviewers)
    if p1==p2:
        return 1., n_common
    #get reviews
    if n_common==0:
        return 0., n_common
    p1_rows = get_restaurant_reviews(p1, df, common_reviewers)
    p2_rows = get_restaurant_reviews(p2, df, common_reviewers)
    sim=similarity_func(p1_rows, p2_rows, n_common)
    return sim, n_common

In [ ]:
def populate_by_calculating(db, df, similarity_func):
    """
    a populator for every pair of businesses in df. takes similarity_func like
    pearson_sim as argument
    """
    items=db.rindexmap.items()
    total = len(items)**2
    count = 0
    start = time.time()
    for b1, i1 in items:
        for b2, i2 in items:
            count += 1
            if (((10000*count)/total) > ((10000*(count-1))/total)):
                frac_done = count/float(total)
                time_elapsed = (time.time() - start) / (60.)
                remaining = ((1./frac_done)-1.)*time_elapsed
                print('%.3f percent done | elapsed: %.1f min | remaining: %.2f hour'%(frac_done *100, time_elapsed, remaining/60))
            if i1 <= i2:
                sim, nsup=calculate_similarity(db, df, b1, b2, similarity_func)
                db.database_sim[i1][i2]=sim
                db.database_sim[i2][i1]=sim
                db.database_sup[i1][i2]=nsup
                db.database_sup[i2][i1]=nsup

In [ ]:
%%time
populate_by_calculating(db2, traindf, pitcher_sim)                

0.010 percent done | elapsed: 0.6 min | remaining: 91.40 hour
0.020 percent done | elapsed: 1.1 min | remaining: 92.23 hour
0.030 percent done | elapsed: 1.9 min | remaining: 106.95 hour
0.040 percent done | elapsed: 2.6 min | remaining: 107.55 hour
0.050 percent done | elapsed: 3.2 min | remaining: 106.71 hour
0.060 percent done | elapsed: 3.7 min | remaining: 102.46 hour
0.070 percent done | elapsed: 4.5 min | remaining: 107.88 hour
0.080 percent done | elapsed: 5.4 min | remaining: 112.36 hour
0.090 percent done | elapsed: 6.7 min | remaining: 123.99 hour
0.100 percent done | elapsed: 8.2 min | remaining: 136.01 hour
0.110 percent done | elapsed: 9.6 min | remaining: 146.00 hour
0.120 percent done | elapsed: 11.6 min | remaining: 160.83 hour
0.130 percent done | elapsed: 13.0 min | remaining: 166.24 hour
0.140 percent done | elapsed: 14.5 min | remaining: 172.58 hour
0.150 percent done | elapsed: 16.0 min | remaining: 177.72 hour

/Users/bcook/anaconda/lib/python2.7/site-packages/IPython/kernel/__main__.py:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/bcook/anaconda/lib/python2.7/site-packages/IPython/kernel/__main__.py:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy



0.160 percent done | elapsed: 17.0 min | remaining: 176.78 hour
0.170 percent done | elapsed: 17.3 min | remaining: 169.68 hour
0.180 percent done | elapsed: 17.5 min | remaining: 161.36 hour
0.190 percent done | elapsed: 17.7 min | remaining: 155.13 hour
0.200 percent done | elapsed: 18.2 min | remaining: 150.97 hour
0.210 percent done | elapsed: 18.6 min | remaining: 147.22 hour
0.220 percent done | elapsed: 18.9 min | remaining: 142.88 hour
0.230 percent done | elapsed: 19.1 min | remaining: 138.38 hour
0.240 percent done | elapsed: 19.3 min | remaining: 133.78 hour
0.250 percent done | elapsed: 19.4 min | remaining: 129.06 hour
0.260 percent done | elapsed: 19.5 min | remaining: 124.71 hour
0.270 percent done | elapsed: 19.7 min | remaining: 120.97 hour
0.280 percent done | elapsed: 19.8 min | remaining: 117.33 hour
0.290 percent done | elapsed: 20.0 min | remaining: 114.38 hour
0.300 percent done | elapsed: 20.1 min | remaining: 111.09 hour
0.310 percent done | elapsed: 20.2 min 

In [56]:
#import cPickle as pickle
#pickle.dump(db2,open('db2.p','wb'))
db2 = pickle.load(open('db2.p','rb'))

In [57]:
smalldf.sort('ovp_FACED').pID.unique()

array(['dotsr001', 'bannf001', 'forsb001', ..., 'hernl003', 'hudst001',
       'buehm001'], dtype=object)

In [58]:
tpair=('buehm001', 'hudst001') # Mark Buehrle vs. Tim Hudson
print db.get(tpair[0],tpair[1])

(0.020463161725546037, 209)


In [ ]:
fig, axis = plt.subplots(figsize=(8,5))
sims = db.database_sim.flatten()
axis.hist(sims[sims>=0], bins=20, normed=True)
axis.set_xlabel('Similarities')

In [ ]:
def shrunk_sim(sim, n_common, reg=3.):
    "takes a similarity and shrinks it down by using the regularizer"
    ssim=(n_common*sim)/(n_common+reg)
    return ssim

In [ ]:
"""
Function
--------
knearest

Parameters
----------
restaurant_id : string
    The id of the restaurant whose nearest neighbors we want
set_of_restaurants : array
    The set of restaurants from which we want to find the nearest neighbors
dbase : instance of Database class.
    A database of similarities, on which the get method can be used to get the similarity
  of two businesses. e.g. dbase.get(rid1,rid2)
k : int
    the number of nearest neighbors desired, default 7
reg: float
    the regularization.
    
  
Returns
--------
A sorted list
    of the top k similar restaurants. The list is a list of tuples
    (business_id, shrunken similarity, common support).
"""
from operator import itemgetter
def knearest(restaurant_id, set_of_restaurants, dbase, k=7, reg=3.):
    """
    Given a restaurant_id, dataframe, and database, get a sorted list of the
    k most similar restaurants from the set of restaurants.
    """
    similars=[]
    for other_rest_id in set_of_restaurants:
        if other_rest_id!=restaurant_id:
            sim, nc=dbase.get(restaurant_id, other_rest_id)
            ssim=shrunk_sim(sim, nc, reg=reg)
            simdist=(1. - ssim)/2.
            similars.append((other_rest_id, simdist, nc ))
    similars=sorted(similars, key=itemgetter(1))
    return similars[0:k]

In [ ]:
def get_users_restaurants(df, b_id):
    dfbatter=df[df.bID==b_id]
    dfbatterdedup=dfbatter.drop_duplicates('pID')
    return dict(zip(dfbatterdedup.pID.values, dfbatterdedup.AVG.values))

In [ ]:
"""
Function
--------
rating

Parameters
----------
set_of_restaurants: Dictionary
    The dictionary of restaurant: star-rating pairs you want to make the prediction from.
    This would be the output of a function like get_users_restaurants
train_map: Dictionary
    A dictionary with keys mean, users and items which have estimates of
    overall average or intercept, user coefficients(averages), and
    item coefficients(averages) respectively
dbase : instance of Database class.
    A database of similarities, on which the get method can be used to get the similarity
  of two businessed. e.g. dbase.get(rid1,rid2)
restaurant_id : string
    The id of the restaurant whose nearest neighbors we want
user_id : string
    The id of the user, in whose reviewed restaurants we want to find the neighbors
k : int
    the number of nearest neighbors desired, default 7
reg: float
    the regularization.
    
  
Returns
--------
A float
    which is the imputed rating that we predict that user_id will make for restaurant_id
    
Notes
--------
If the sum of scores is 0, return the baseline estimate of the ranking.
"""
#your code here
# Note: this function was inspired in part by the solutions to the 2013 hw4
def rating(set_of_restaurants, train_map, dbase, restaurant_id, user_id, k=7, reg=3.):
    mu=train_map['mean']
    user_bias = train_map['batters'][user_id]
    nsum=0.
    scoresum=0.
    nears=knearest(restaurant_id, set_of_restaurants, dbase, k=k, reg=reg)
    restaurant_bias=train_map['pitchers'][restaurant_id]
    scores=[]
    for r,s,nc in nears:
        ssim = 1-s
        scoresum=scoresum+ssim
        scores.append(ssim)
        r_biases = train_map['pitchers'][r]
        r_stars = set_of_restaurants[r]
        rminusb=(r_stars - (r_biases + user_bias + mu))
        nsum=nsum+ssim*rminusb
    baseline=(user_bias +restaurant_bias + mu)
    if scoresum > 0.:
        val =  nsum/scoresum + baseline
    else:
        val=baseline
    return val

In [ ]:
trainuser=traindf.loc[100].bID
try:
    testrest=testdf[testdf.bID==trainuser].pID.values[0]
except:
    testrest=testdf[testdf.bID==trainuser].pID
print trainuser, testrest

In [ ]:
testdf[testdf.bID=='abreb001'].pID.values

In [ ]:
traindf.head()

In [ ]:
def get_actual(df, userid, bizid):
    return df[(df.bID==userid) & (df.pID==bizid)]['AVG'].values[0]

In [ ]:
print "Actual", get_actual(testdf, trainuser, testrest)
print "Predicted",rating(get_users_restaurants(traindf, trainuser), train_avgs, db, testrest, trainuser, k=2, reg=3.)

In [ ]:
def get_ratings_user_nbd(indf, traindf, train_map, db, k=2, reg=3.):
    zips=zip(indf.pID, indf.bID, indf.AVG)
    preds=[]
    actuals=[]
    for (r,u,actual) in zips:
        pred=rating(get_users_restaurants(traindf, u),train_map, db, r,u, k, reg)
        preds.append(pred)
        actuals.append(actual)
    return np.array(preds), np.array(actuals)

In [ ]:
%%time
pt, at = get_ratings_user_nbd(traindf, traindf, train_avgs, db, k=6, reg=4.)
compare_results(at,pt, model="knn(user) on training k=6, reg=4", predicteds=True)

In [ ]:
%%time
k = range(40,100,10)
reg = range(1,12,2)
rmsedict = {}
for kval in k:
    for regval in reg:
        pt, at = get_ratings_user_nbd(validatedf, traindf, train_avgs, db, k=kval, reg=regval)
        rmse = get_rmse(at, pt)
        key = (kval, regval)
        rmsedict[key] = rmse
   

In [ ]:
     
mintup=min(rmsedict, key=rmsedict.get)
mintup

In [ ]:
bestk, bestreg = mintup
predictions["knn"], atknn = get_ratings_user_nbd(testdf, traindf,train_avgs, db, k=bestk, reg=bestreg)
predictions_valid["knn"], atvalid = get_ratings_user_nbd(traindf, traindf, train_avgs, db, k=bestk, reg=bestreg)

In [ ]:
ax, rmse_knn = compare_results(testdf.AVG, predictions['knn'], model=("knn(user) on test k=" + str(bestk) + ", reg=" + str(bestreg)), predicteds=True)
# compare_results(testdf.stars,predictions['baseline'], model="baseline", predicteds=True, onsame=ax)